<a href="https://colab.research.google.com/github/LWS49/Hackathon-Submissions/blob/main/Student_DeliveRHy_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The bot's username is @DeliverhyBot. The bot can only be used when this programme is running.

To access the bot, search for @DeliverhyBot.

In [ ]:
#######################################
###### BASIC INFRASTRUCTURE ###########
#######################################

import logging
import math
import telegram
from random import random
from telegram import Update, ForceReply, ReplyKeyboardMarkup
from datetime import datetime

from telegram import Update, ForceReply
from telegram.ext import (Updater, 
                          CommandHandler, 
                          ConversationHandler, 
                          MessageHandler, 
                          Filters, 
                          CallbackContext)

logging.basicConfig(
    format='%(asctime)s - %(name)s - $(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

token = '5920972783:AAEsxLDaHIJKBKDYnKKp1oMaPjyuxve_sL4'

bot = telegram.Bot(token='5804374260:AAHlHmh9QYnpUXGqDOBKkLDzA_YCnsI-7NA')
def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s caused error "%s', update, context.error)

In [ ]:
#################################
### GLOBAL VARIABLES ############
#################################

DEFAULT_LOCATION = ["YIH", "UTOWN", "TECHNOEDGE"] # the locations that are open for delivery, up for modifications

(USERTYPE, DISCLAIMER, CLOSINGTIME, REACHTIME, LOCATION, REMOVE, 
 ADMINLOGIN, ADMINFUNCTIONS, CHARGE, CHECKADDCOMMENTS, ADDCOMMENTS, MESSAGE1) = range(12)

TypeOfUser = {"I would like to deliver":"deliverer", "I would like to request a delivery": "receiver"}
class Person:
  def __init__(self, is_deliverer, handle, closetime, reachtime, location, charge, comments, userid):
    self.handle = handle
    self.is_deliverer = is_deliverer
    self.closetime = closetime
    self.reachtime = reachtime
    self.location = location
    self.comments = comments
    self.charge = charge
    self.userid = userid

person = Person(False, "non-existent", "2400", "2400", "RH", "0.50", "NA", 12345) 
# this dummy variable will store the information of the deliverer

criteria_for_filtering = [False, "2400", "RH"] # dummy variable for /show function

deliverers_by_location = {"YIH":[],"UTOWN":[],"TECHNOEDGE":[]}
# deliverers list sorted by location (to cater to large numbers when sorting)
deliverers_by_closingtime = {}
deliverers_by_handle = {}

bot_paused = [False] # to check if the bot is paused

In [ ]:
#########################
### ADMIN OPERATIONS ####
#########################
admin_dict = {"admin1": "deliveRHy"} # admin accounts for resettting

# Admin function description
# 1: Stop bot: Pauses the bot. No one can edit the lists. Can be useful if bug fixing is needed.
# 2: Resume bot: Resumes the bot.
# 3: Clear list: Clears the list manually. Can be useful in bug fixing too, or if there is too much spam.

def admin(update: Update, _: CallbackContext) -> None:
    update.message.reply_text('Please enter your username and password.')
    return ADMINLOGIN

def adminlogin(update: Update, _: CallbackContext) -> None:
    adminstring = update.message.text
    username = adminstring.split()[0]
    password = adminstring.split()[1]
    if username in admin_dict.keys() and admin_dict[username] == password:
      keyboard = [["Stop bot", "Resume bot", "Clear lists", "NA"]]
      reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
      update.message.reply_text('What would you like to do? ', reply_markup=reply_markup)
      return ADMINFUNCTIONS
    else:
      update.message.reply_text('Invalid combination. Type /admin to try again.')
      return ConversationHandler.END

def adminfunctions(update: Update, _: CallbackContext) -> None:
    function = update.message.text
    if function == "NA":
      update.message.reply_text('Have a great day!')
      return ConversationHandler.END
    elif function == "Clear lists":
      clear_all()
      update.message.reply_text('Lists cleared.')
      return ConversationHandler.END
    elif function == "Stop bot":
      bot_paused[0] = True
      update.message.reply_text('Bot is paused.')
      print(bot_paused) ##
      return ConversationHandler.END
    else:
      bot_paused[0] = False
      update.message.reply_text('Bot is resumed.')
      return ConversationHandler.END

In [ ]:
############################
### INITIAL COMMANDS #######
############################

# Admin function description
# 1: /info: Provides info on the bot.
# 2: /start: Starts the conversation for the user.
# 3: /remove: Remove the telegram handle user from lists. For if deliverers 
#             who change their mind / want to ammend the details.
#             - to prevent user from registering or removing other users, 
#               users can only add or remove their own telehandles from the list.
# 4: /show: Show the number of deliverers available before a time for a location.
#           - More convenient for users to decide where they would like to order from, 
#             depending on availability.

def info(update: Update, _: CallbackContext) -> None:
    update.message.reply_text('Hi, thanks for using Student DeliveRHy! \n ' + 
    'The following commands can be used with this bot:\n' +
    '/start to get started on indicating interest in delivering or ordering a delivery\n' +
    '/remove to remove yourself from the list of people interested in delivering\n' + 
    '/admin for admin functions\n')

def start(update: Update, _: CallbackContext) -> None:
    if bot_paused[0]:
      update.message.reply_text('The bot has been paused. The bot will only resume' +
      'after an admin re-enables it. We thank your patience in this matter.')
      return ConversationHandler.END
    else:
     keyboard = [["I would like to deliver", "I would like to request a delivery"]]
     reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
     update.message.reply_text('Hi, thanks for using Student DeliveRHy! Would ' + 
     'you like to volunteer to deliver or ' +
     'find someone to help deliver food to RH for you?', reply_markup=reply_markup)
     return USERTYPE

def remove(update: Update, _: CallbackContext) -> None:
    if bot_paused[0]:
      update.message.reply_text('The bot has been paused. The bot will only resume' +
      'after an admin re-enables it. We thank your patience in this matter.')
      return ConversationHandler.END
    handle = update.message.from_user.username
    keyboard = [["Yes", "No"]]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    update.message.reply_text('Hi, thanks for using Student DeliveRHy! \n\n' + 
    f'Your handle {handle} will be removed from all current lists. \n' +
    'Click Yes to proceed.', reply_markup=reply_markup)
    return REMOVE

def show(update: Update, _: CallbackContext) -> None:
    criteria_for_filtering[0] = True
    update.message.reply_text('Hi, thanks for using Student DeliveRHy! \n' +
      'When is the latest you would like your food to arrive at RH? \n' +
      '\n Please indicate the timing in 24-hour format, e.g. 1300')
    return REACHTIME

In [ ]:
#####################################
######## HELPER FUNCTIONS ###########
#####################################

# Helper function description
# 1. filt: filter function for "receivers" to see the list of deliverers the user can message
# 2. clear: clears the list of deliverers
# 3. update_and_craft: Update list of deliverers and return the final message
# 4. message_crafting: Craft list of deliverers after given location and time

def filt(loc, rchtime): 
  lst=[]
  for i in deliverers_by_location[loc]:
    if int(i.reachtime)<=int(rchtime):
      lst.append((i.handle, i.charge, i.comments),)
  return lst

def filt2(l):
  handle_list = []
  for i in l:
    handle_list.append(i[0])
  return handle_list

def clear_all():
  for i in deliverers_by_location:
   deliverers_by_location[i].clear()
  deliverers_by_handle.clear()
  deliverers_by_closingtime.clear()

# clears the list of deliverers

# usertype -> disclaimer -> charge -> closingtime -> rchtime 
# -> location -> additional comments? -> additional comments

def update_and_craft():
  deliverers_by_location[person.location].append(person)
  closinghr = str(int(person.closetime[:2]))
  if closinghr in deliverers_by_closingtime.keys():
    deliverers_by_closingtime[closinghr].append(person)
  else:
    deliverers_by_closingtime[closinghr] = person
  if person.handle in deliverers_by_handle.keys():
    deliverers_by_handle[person.handle].append(person)
  else:
    deliverers_by_handle[person.handle] = [person]
  details = "\nHandle: @" + person.handle + '\nCharge: $' + person.charge + '\nOrder Closing Time: ' \
  + person.closetime + '\nReaching RH by: ' + person.reachtime + '\nLocation: ' \
  + person.location + '\nAdditional Comments: ' + person.comments
  msg = ('Thank you for using Student DeliveRHy! The following are the details you submitted:\n' 
  + details + '\n\n\nInterested RHesidents would contact you via your telegram.' +
  f'\nYour telegram handle will be removed from the list of those available at {person.closetime}.\n' + 
  '\nIf you entered the details wrongly, remember that you can always remove ' + 
  'yourself from the list using /remove and re-inputing again. Thank you!')
  return msg


def message_crafting(lst):
  text = 'The handles of people that are available for delivery and their charges are as follows: \n'
  for deliverer in lst:
    if deliverer[2] == "NA":
      add_on = '@' + deliverer[0] + " $" + deliverer[1] + '\n'
    else: 
      add_on = '@' + deliverer[0] + " $" + deliverer[1] + "   '" + deliverer[2] + "'\n"
    text += add_on
  return text

def hourly_removal(hr):
    for i in deliverers_by_location:
          for j in deliverers_by_location[i]:
            if j.closetime == hr:
              deliverers_by_location[i].remove(j)
    if hr in deliverers_by_closingtime.keys():
      del deliverers_by_closingtime[hr]

In [ ]:
#################################
##### COMMANDS HANDLING STATE ###
#################################

# Flow of conversation:
# For deliverers: usertype -> disclaimer ->f charge -> closingtime -> rchtime 
# -> location -> additional comments? -> additional comments
# For those who want delivery: usertype -> rchtime -> location.
# Information is obtained in this order.

def usertype(update: Update, context: CallbackContext) -> None:
    Type = update.message.text
    if TypeOfUser[Type] == "deliverer":
      person.is_deliverer = True
      keyboard = [["Yes", "No"]]
      reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
      update.message.reply_text(f'Your telegram handle is @{update.message.from_user.username}' + \
      '. Do you consent for this telegram handle to be sent to other users' + \
      ' so that they can contact you for delivery?', reply_markup=reply_markup)
      return DISCLAIMER # check for consent to send handle
    else:
      person.is_deliverer = False
      update.message.reply_text('When is the latest you would like your food to arrive at RH?' + \
      '\nPlease indicate the timing in 24-hour format, e.g. 1300')
      return REACHTIME

def disclaimer(update: Update, context: CallbackContext) -> None:
    consent = update.message.text
    if consent == "Yes":
      person.handle = update.message.from_user.username
      person.userid = update.message.from_user.id
      update.message.reply_text('How much would you like to charge?' + \
      '\nPlease type the amount in dollars without the dollar sign, e.g. "0.50" for 50 cents.')
      return CHARGE
    else: 
      update.message.reply_text('Unfortunately, we require those who are interested' +
      ' in delivering to provide their telehandle. We thank you for your interest' +
      ' in volunteering to deliver. Have a great day!')
      return ConversationHandler.END

def charge(update: Update, context: CallbackContext) -> None:
    charge = update.message.text
    try:
      charge = float(charge) # check if input is integer or decimal number - reject if not both
    except:
      update.message.reply_text('Error: Please type the amount in dollars without the' + 
      ' dollar sign, e.g. "0.50" for 50 cents.')
      return CHARGE # loop if input is not integer or decimal number
    person.charge = str(charge)
    update.message.reply_text('When is the latest you would like to close your order?' + \
      '\nPlease indicate the timing in 24-hour format, e.g. 1300')
    return CLOSINGTIME

def closingtime(update: Update, context: CallbackContext) -> None:
    CTime = update.message.text 
    CTime = str(CTime)
    if (not CTime.isdigit() or len(CTime) != 4 or int(CTime[-2:]) > 59 or 
        int(CTime[-2:]) < 00 or int(CTime[:2]) > 23 or int(CTime[:2]) < 0):
        # check if input is time in 24-hour
        update.message.reply_text('Error: Please indicate the timing in 24-hour format, e.g. 1300')
        return CLOSINGTIME # loop if input is not time in 24-hour
    person.closetime = CTime
    update.message.reply_text('When is the latest you would arrive at RH?' + \
    '\n Please indicate the timing in 24-hour format, e.g. 1300')
    return REACHTIME

def rchtime(update: Update, context: CallbackContext) -> None:
    RTime = update.message.text
    if (not RTime.isdigit() or len(RTime) != 4 or int(RTime[-2:]) > 59 or 
        int(RTime[-2:]) < 00 or int(RTime[:2]) > 23 or int(RTime[:2]) < 0):
        # check if input is time in 24-hour
        update.message.reply_text('Error: Please indicate the timing in 24-hour format, e.g. 1300')
        return REACHTIME # loop if input is not time in 24-hour
    if criteria_for_filtering[0]: 
      criteria_for_filtering[1] = RTime
    else:
      person.reachtime = RTime
    keyboard = [DEFAULT_LOCATION]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    if person.is_deliverer:
      update.message.reply_text('Where would you be delivering from?', reply_markup=reply_markup)
    else:
      update.message.reply_text('Where would you like your food from?', reply_markup=reply_markup)
    return LOCATION

def location(update: Update, context: CallbackContext) -> None:
    loc = update.message.text
    if criteria_for_filtering[0]: 
      lst = filt(loc, criteria_for_filtering[1])
      update.message.reply_text(f'The number of people available to deliver for you: {len(lst)}')
      return ConversationHandler.END
    else:
      person.location = loc
    if person.is_deliverer:
      keyboard = [["Yes", "No"]]
      reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
      update.message.reply_text('Would you like to add any additional short comments?', reply_markup=reply_markup)
      return CHECKADDCOMMENTS
    else: 
      lst = filt(person.location, person.reachtime)
      message_text = message_crafting(lst) # compile list of deliverers (under "HELPER FUNCTIONS")
      update.message.reply_text(message_text) 
      handle_list = filt2(lst)
      keyboard = [handle_list]
      keyboard.append('X')
      reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
      update.message.reply_text('Which user do you prefer? ' +
                                'Press X to cancel', reply_markup=reply_markup)
      return MESSAGE1

def message1(update: Update, _: CallbackContext) -> None:
  temp = update.message.text
  selected = 1
  if temp == "X":
    return ConversationHandler.END
  else: 
    for x in deliverers_by_handle:
        print(x)
        if x == temp:
          selected = deliverers_by_handle[x][0]
          selected_id = selected.userid
          selected_user = selected.handle 
          selected_close = selected.closetime
        else:
          x = None
    bot.send_message(chat_id=selected_id, text='Someone has expressed interest, ' +
                   'look out for their messages!')
    update.message.reply_text(f"We have alerted {selected_user}, " +
                            f"Don't forget to make contact by {selected_close}!")
    return ConversationHandler.END

def checkaddcomments(update: Update, context: CallbackContext) -> None:
    temp = update.message.text
    if temp == "Yes":
      update.message.reply_text('Please type your additional comments. You are' +
      ' limited to 20 charcters')
      return ADDCOMMENTS
    else:
      person.comments = "NA"
      message = update_and_craft() # update & compile final message (under "HELPER FUNCTIONS")
      update.message.reply_text(message) 
      return ConversationHandler.END

def addcomments(update: Update, context: CallbackContext) -> None:
    comm = update.message.text
    if (len(comm) > 20):
      update.message.reply_text('Your comment is too long. Please type a comment' +
      ' with less than 20 charcters')
      return ADDCOMMENTS
    person.comments = comm
    message = update_and_craft() # update & compile final message (under "HELPER FUNCTIONS")
    update.message.reply_text(message)
    return ConversationHandler.END
      

def remove2(update: Update, _: CallbackContext) -> None:
    temp = update.message.text
    if temp == "Yes":
      handle = update.message.from_user.username
      for i in deliverers_by_location:
          for j in deliverers_by_location[i]:
            if j.handle==handle:
              deliverers_by_location[i].remove(j)
      for i in deliverers_by_closingtime:
          for j in deliverers_by_closingtime[i]:
            if j.handle==handle:
              deliverers_by_location[i].remove(j)
      del deliverers_by_handle[handle] # remove them from all 3 lists
      update.message.reply_text('Your handle has been removed from all current lists.' 
      + 'We thank you for your interest in Student DeliveRHy.')
    else:
      update.message.reply_text('The removal is unsuccessful.')
    return ConversationHandler.END
# remove2 is to confirm that they do want to remove themselves from the list 
# before doing so and it's not a misclick

In [ ]:
# Command to cancel ConversationHandler
def cancel(update: Update, _: CallbackContext) -> None:
    update.message.reply_text('Please try again.')


In [ ]:
#################################
######### BOT SETUP #############
################################

def main() -> None:

    # Create the Updater and pass it to your bot's token.
    updater = Updater(token)

    # Get the dispatcher to register handlers.
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    # send '/start'

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start), 
            CommandHandler('remove', remove),
            CommandHandler('admin', admin),
            CommandHandler('show', show),
            ],
        states={
            USERTYPE: [MessageHandler(Filters.text, usertype)],
            LOCATION: [MessageHandler(Filters.text, location)],
            DISCLAIMER: [MessageHandler(Filters.text, disclaimer)],
            REACHTIME: [MessageHandler(Filters.text, rchtime)],
            CLOSINGTIME: [MessageHandler(Filters.text, closingtime)],
            ADMINLOGIN: [MessageHandler(Filters.text, adminlogin)],
            ADMINFUNCTIONS: [MessageHandler(Filters.text, adminfunctions)],
            REMOVE: [MessageHandler(Filters.text, remove2)],
            CHARGE: [MessageHandler(Filters.text, charge)],
            CHECKADDCOMMENTS: [MessageHandler(Filters.text, checkaddcomments)],
            ADDCOMMENTS: [MessageHandler(Filters.text, addcomments)],
            MESSAGE1: [MessageHandler(Filters.text, message1)]
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(conv_handler)

    # Start the Bot
    updater.start_polling()

    # j = updater.job_queue
    # job_daily = j.run_daily(clear, time = datetime.time(00, 00, 00))
    # clear the list once daily
    # currenthr = str((datetime.now().hour + 7) % 24)
    # print(currenthr)
    # remove_expired_deliverers = j.run_repeating(hourly_removal(currenthr), 3600)
    # clear the list every hour

    # Run the bot until you press Ctrl-C or the process receives SIGINT, 
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blockng and will stop the bot gracefully.
    updater.idle()

In [ ]:
if __name__ == '__main__':
    main()